THIS IS OUR GROUP PROJECT YAY
Code is from:
https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

In [4]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [5]:
#WRITE TOKEN HERE
os.environ['TOKEN'] =

In [6]:
def auth():
    return os.getenv('TOKEN')

In [7]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [119]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

#CHANGE THIS PART TO DISPLAY DIFFERENT VALUES
    query_params = {'query': keyword,
                    'max_results': max_results,
                    'expansions': 'author_id,geo.place_id', #'author_id,in_reply_to_user_id,geo.place_id', expansions controls which of the below fields you want to see
                    'tweet.fields': 'text,public_metrics,referenced_tweets,geo,created_at', #'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'name,username,created_at,description,public_metrics,verified', #'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,country,country_code,geo,name,place_type', #'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [120]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [121]:
#CHANGE
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "UCLA" #if user, from:, #, ucla
start_time = "2022-02-01T00:00:00.000Z"
end_time = "2022-02-28T00:00:00.000Z"
max_results = 100

In [122]:
url = create_url(keyword, start_time,end_time, max_results)
print(url)
json_response = connect_to_endpoint(url[0], headers, url[1])

('https://api.twitter.com/2/tweets/search/recent', {'query': 'UCLA', 'max_results': 100, 'expansions': 'author_id,geo.place_id', 'tweet.fields': 'text,public_metrics,referenced_tweets,geo,created_at', 'user.fields': 'name,username,created_at,description,public_metrics,verified', 'place.fields': 'full_name,country,country_code,geo,name,place_type', 'next_token': {}})
Endpoint Response Code: 200


In [123]:
json_response

{'data': [{'referenced_tweets': [{'type': 'retweeted',
     'id': '1500304222998319107'}],
   'text': 'RT @UCLASoftball: 💥 𝐎𝐅𝐅𝐄𝐍𝐒𝐈𝐕𝐄 𝐄𝐗𝐏𝐋𝐎𝐒𝐈𝐎𝐍 💥\n\nSavi Pola leads the Bruins with 6 RBI in her 3-for-4 day (2 2B, HR), Delanie Wisz goes deep TWIC…',
   'created_at': '2022-03-06T09:20:23.000Z',
   'id': '1500400892591411203',
   'public_metrics': {'retweet_count': 32,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '1418429823920316422'},
  {'referenced_tweets': [{'type': 'retweeted', 'id': '1500221911187562498'}],
   'text': 'RT @dr_raddatz: Jetzt kommt alles ans Licht über Nebenwirkungen der BionTech/Pfizer Giftspritze:\n1/3 "plaintiffs, a group of more than 30 p…',
   'created_at': '2022-03-06T09:20:18.000Z',
   'id': '1500400871636705282',
   'public_metrics': {'retweet_count': 152,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '811135456100356096'},
  {'referenced_tweets': [{'type': 'retweeted', 'id': '150022191118

Change JSON to dataframe 

In [124]:
with open('imadejson.json', 'w') as outfile:
     json.dump(json_response, outfile)

In [125]:
#CHANGE THIS TO WHERE YOUR FILE IS LOCATED
file = json.load(open(r'C:\Users\melinadiaz\imadejson.json'))

tweets = pd.DataFrame(file["data"])
users = pd.DataFrame(file["includes"]["users"])
places = pd.DataFrame(file["includes"]["places"])

In [126]:
tweets

,referenced_tweets,text,created_at,id,public_metrics,author_id,geo
0,"[{'type': 'retweeted', 'id': '1500304222998319...",RT @UCLASoftball: 💥 𝐎𝐅𝐅𝐄𝐍𝐒𝐈𝐕𝐄 𝐄𝐗𝐏𝐋𝐎𝐒𝐈𝐎𝐍 💥\n\nS...,2022-03-06T09:20:23.000Z,1500400892591411203,"{'retweet_count': 32, 'reply_count': 0, 'like_...",1418429823920316422,NaN
1,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:20:18.000Z,1500400871636705282,"{'retweet_count': 152, 'reply_count': 0, 'like...",811135456100356096,NaN
2,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:20:09.000Z,1500400832742891526,"{'retweet_count': 152, 'reply_count': 0, 'like...",1449788333031313410,NaN
3,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:20:00.000Z,1500400795497418753,"{'retweet_count': 152, 'reply_count': 0, 'like...",598608687,NaN
4,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:19:02.000Z,1500400553469304834,"{'retweet_count': 152, 'reply_count': 0, 'like...",1053321715160354817,NaN
...,...,...,...,...,...,...,...
95,"[{'type': 'retweeted', 'id': '1499949474726166...",RT @SandraEssays: #GramFam #HBCU #ASUTwitter #...,2022-03-06T08:39:21.000Z,1500390564591804416,"{'retweet_count': 5, 'reply_count': 0, 'like_c...",1331581533468172293,NaN
96,"[{'type': 'retweeted', 'id': '1500298988800643...",RT @SandraEssays: Need help completing your as...,2022-03-06T08:39:09.000Z,1500390517556887556,"{'retweet_count': 4, 'reply_count': 0, 'like_c...",1331581533468172293,NaN
97,"[{'type': 'retweeted', 'id': '1498983619062218...",RT @solobanton_desu: 名門UCLAの体操選手が往年のダンスホールレゲエの...,2022-03-06T08:38:15.000Z,1500390290921648129,"{'retweet_count': 3342, 'reply_count': 0, 'lik...",2482162981,NaN
98,"[{'type': 'retweeted', 'id': '1500369563929567...",RT @voteblew: @IAPonomarenko The talented Ukra...,2022-03-06T08:37:51.000Z,1500390188652146692,"{'retweet_count': 129, 'reply_count': 0, 'like...",432883951,NaN


In [127]:
users

,name,public_metrics,description,created_at,username,verified,id
0,Antonio ⚔️,"{'followers_count': 155, 'following_count': 86...",,2021-07-23T04:37:01.000Z,xAD4_,False,1418429823920316422
1,MapelleFluse,"{'followers_count': 974, 'following_count': 79...","Danke, mir reicht der gesunde Menschenverstand!",2016-12-20T09:05:37.000Z,Tgoli2,False,811135456100356096
2,Jane Doe,"{'followers_count': 1546, 'following_count': 1...","Radikalisierte, maßnahmekritische, nichtzuimpf...",2021-10-17T17:24:45.000Z,JaneDoow,False,1449788333031313410
3,🅲🅾🅾🅿🅴🆁,"{'followers_count': 236, 'following_count': 83...",,2012-06-03T17:58:19.000Z,nerd_hero,False,598608687
4,oliva,"{'followers_count': 1798, 'following_count': 1...",ᴘᴇᴍʙᴇ ᴏʟsᴀ ᴅᴀ ᴋɪᴍʟɪğɪᴍ ᴍᴀᴠɪ' ᴅɪʀ ʙᴇɴɪᴍ ʏüʀᴇğɪᴍ...,2018-10-19T16:27:21.000Z,oliva57172488,False,1053321715160354817
...,...,...,...,...,...,...,...
85,Media Rosey,"{'followers_count': 580, 'following_count': 24...","Media and Marketing Pro, All things SDSU & MWC",2009-04-15T07:02:21.000Z,mediarosey,False,31360112
86,Damocles,"{'followers_count': 1770, 'following_count': 2...",,2020-11-06T08:18:31.000Z,Damocle63375356,False,1324627158514487300
87,ruitaro,"{'followers_count': 2538, 'following_count': 2...",ゆるゆる生きております。🐷ぶひ。No DM！ アイコンは阿信🇹🇼,2014-05-07T13:47:41.000Z,ruinojyo55,False,2482162981
88,caterina lubrano,"{'followers_count': 6, 'following_count': 43, ...","casalinga, amo la mia famiglia,odio l'ingiusti...",2011-12-09T22:03:45.000Z,catvitlove,False,432883951


In [128]:
places

,full_name,id,country_code,geo,country,name,place_type
0,"Los Angeles, CA",3b77caf94bfc81fe,US,"{'type': 'Feature', 'bbox': [-118.668404, 33.7...",United States,Los Angeles,city


IDK why `places` is short. My guess is because most of the tweets aren't associated with a location hence the NaN in the geo column (like the author turned location off), and if location is on, the tweets available are only from LA.

I'm assuming users is generated based off the authors of the tweets, but I can check this because `tweets` has a column of author_id. This should match the id in `users`.

In [129]:
users["author_id"]=users["id"]

In [130]:
tweetsandusers=pd.merge(tweets,users, on="author_id")
tweetsandusers

,referenced_tweets,text,created_at_x,id_x,public_metrics_x,author_id,geo,name,public_metrics_y,description,created_at_y,username,verified,id_y
0,"[{'type': 'retweeted', 'id': '1500304222998319...",RT @UCLASoftball: 💥 𝐎𝐅𝐅𝐄𝐍𝐒𝐈𝐕𝐄 𝐄𝐗𝐏𝐋𝐎𝐒𝐈𝐎𝐍 💥\n\nS...,2022-03-06T09:20:23.000Z,1500400892591411203,"{'retweet_count': 32, 'reply_count': 0, 'like_...",1418429823920316422,NaN,Antonio ⚔️,"{'followers_count': 155, 'following_count': 86...",,2021-07-23T04:37:01.000Z,xAD4_,False,1418429823920316422
1,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:20:18.000Z,1500400871636705282,"{'retweet_count': 152, 'reply_count': 0, 'like...",811135456100356096,NaN,MapelleFluse,"{'followers_count': 974, 'following_count': 79...","Danke, mir reicht der gesunde Menschenverstand!",2016-12-20T09:05:37.000Z,Tgoli2,False,811135456100356096
2,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:20:09.000Z,1500400832742891526,"{'retweet_count': 152, 'reply_count': 0, 'like...",1449788333031313410,NaN,Jane Doe,"{'followers_count': 1546, 'following_count': 1...","Radikalisierte, maßnahmekritische, nichtzuimpf...",2021-10-17T17:24:45.000Z,JaneDoow,False,1449788333031313410
3,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:20:00.000Z,1500400795497418753,"{'retweet_count': 152, 'reply_count': 0, 'like...",598608687,NaN,🅲🅾🅾🅿🅴🆁,"{'followers_count': 236, 'following_count': 83...",,2012-06-03T17:58:19.000Z,nerd_hero,False,598608687
4,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T09:19:02.000Z,1500400553469304834,"{'retweet_count': 152, 'reply_count': 0, 'like...",1053321715160354817,NaN,oliva,"{'followers_count': 1798, 'following_count': 1...",ᴘᴇᴍʙᴇ ᴏʟsᴀ ᴅᴀ ᴋɪᴍʟɪğɪᴍ ᴍᴀᴠɪ' ᴅɪʀ ʙᴇɴɪᴍ ʏüʀᴇğɪᴍ...,2018-10-19T16:27:21.000Z,oliva57172488,False,1053321715160354817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"[{'type': 'retweeted', 'id': '1500369563929567...",RT @voteblew: @IAPonomarenko The talented Ukra...,2022-03-06T08:40:16.000Z,1500390797132197890,"{'retweet_count': 129, 'reply_count': 0, 'like...",31360112,NaN,Media Rosey,"{'followers_count': 580, 'following_count': 24...","Media and Marketing Pro, All things SDSU & MWC",2009-04-15T07:02:21.000Z,mediarosey,False,31360112
96,"[{'type': 'retweeted', 'id': '1500221911187562...",RT @dr_raddatz: Jetzt kommt alles ans Licht üb...,2022-03-06T08:39:51.000Z,1500390691058376705,"{'retweet_count': 152, 'reply_count': 0, 'like...",1324627158514487300,NaN,Damocles,"{'followers_count': 1770, 'following_count': 2...",,2020-11-06T08:18:31.000Z,Damocle63375356,False,1324627158514487300
97,"[{'type': 'retweeted', 'id': '1498983619062218...",RT @solobanton_desu: 名門UCLAの体操選手が往年のダンスホールレゲエの...,2022-03-06T08:38:15.000Z,1500390290921648129,"{'retweet_count': 3342, 'reply_count': 0, 'lik...",2482162981,NaN,ruitaro,"{'followers_count': 2538, 'following_count': 2...",ゆるゆる生きております。🐷ぶひ。No DM！ アイコンは阿信🇹🇼,2014-05-07T13:47:41.000Z,ruinojyo55,False,2482162981
98,"[{'type': 'retweeted', 'id': '1500369563929567...",RT @voteblew: @IAPonomarenko The talented Ukra...,2022-03-06T08:37:51.000Z,1500390188652146692,"{'retweet_count': 129, 'reply_count': 0, 'like...",432883951,NaN,caterina lubrano,"{'followers_count': 6, 'following_count': 43, ...","casalinga, amo la mia famiglia,odio l'ingiusti...",2011-12-09T22:03:45.000Z,catvitlove,False,432883951


Since this df is 100 rows, the same as the tweets, I think this means every tweet has a user that was in `users`. Next step is maybe filtering the tweets. Like looking at tweets with at least 10 likes, or looking at not retweeted tweets.